# 1. Handlinng missing data

In [3]:
import numpy as np
import pandas as pd

### Pandas utility functions
Similarly to numpy, pandas also has a few utility functions to identify and detect null values:

In [4]:
pd.isnull(np.nan)

True

In [5]:
pd.isnull(None)

True

In [6]:
pd.isna(np.nan) #null and na is the same in pandas

True

In [7]:
pd.isna(None)

True


The opposite ones also exist:

In [8]:
pd.notnull(None)

False

In [9]:
pd.notnull(np.nan)

False

In [10]:
pd.notna(np.nan)

False

In [11]:
pd.notnull(3)

True

These functions also work with Series and DataFrames:

In [12]:
pd.isnull(pd.Series([1, np.nan, 7]))

0    False
1     True
2    False
dtype: bool

In [13]:
pd.notnull(pd.Series([1, np.nan, 7]))

0     True
1    False
2     True
dtype: bool

In [14]:
pd.isnull(pd.DataFrame({
    'Column A': [1, np.nan, 7],
    'Column B': [np.nan, 2, 3],
    'Column C': [np.nan, 2, np.nan]
}))

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


### Pandas Operations with Missing Values
Pandas manages missing values more gracefully than numpy. nans will no longer behave as "viruses", and operations will just ignore them completely:

In [15]:
pd.Series([1, 2, np.nan]).count()

2

In [16]:
pd.Series([1, 2, np.nan]).sum()

3.0

In [17]:
pd.Series([2, 2, np.nan]).mean()

2.0

### Filtering missing data
As we saw with numpy, we could combine boolean selection + pd.isnull to filter out those nans and null values:

In [18]:
s = pd.Series([1, 2, 3, np.nan, np.nan, 4])

In [19]:
pd.notnull(s)

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool

In [20]:
pd.isnull(s)

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

In [21]:
pd.notnull(s).sum()

4

In [22]:
pd.isnull(s).sum()

2

In [23]:
s[pd.notnull(s)]  #filtering the values with a series

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

In [24]:
s.isnull()

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

In [25]:
s.notnull()

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool

In [26]:
s[s.notnull()] 

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

### Dropping null values
Boolean selection + notnull() seems a little bit verbose and repetitive. And as we said before: any repetitive task will probably have a better, more DRY way. In this case, we can use the dropna method:

In [28]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

In [29]:
s.dropna() #Dropping null values ROWS

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

## Dropping null values on DataFrames
You saw how simple it is to drop nas with a Series. But with DataFrames, there will be a few more things to consider, because you can't drop single values. You can only drop entire columns or rows. Let's start with a sample DataFrame:

In [30]:
df = pd.DataFrame({
    'Column A': [1, np.nan, 30, np.nan],
    'Column B': [2, 8, 31, np.nan],
    'Column C': [np.nan, 9, 32, 100],
    'Column D': [5, 8, 34, 110],
})

In [31]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [32]:
df.shape

(4, 4)

In [38]:
df.info() # veru useful!!!!!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Column A  2 non-null      float64
 1   Column B  3 non-null      float64
 2   Column C  3 non-null      float64
 3   Column D  4 non-null      int64  
dtypes: float64(3), int64(1)
memory usage: 256.0 bytes


In [34]:
df.isnull()

,Column A,Column B,Column C,Column D
0,False,False,True,False
1,True,False,False,False
2,False,False,False,False
3,True,True,False,False


In [35]:
df.isnull().sum()

Column A    2
Column B    1
Column C    1
Column D    0
dtype: int64


The default dropna behavior will drop all the rows in which any null value is present:  DROPPING ROWS!!!

In [36]:
df.dropna()

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


In this case we're dropping rows. Rows containing null values are dropped from the DF. You can also use the axis parameter to drop columns containing null values:

In [37]:
df.dropna(axis=1)  # axis='columns' also works

,Column D
0,5
1,8
2,34
3,110


In this case, any row or column that contains at least one null value will be dropped. Which can be, depending on the case, too extreme. You can control this behavior with the how parameter. Can be either 'any' or 'all':

In [39]:
df2 = pd.DataFrame({
    'Column A': [1, np.nan, 30],
    'Column B': [2, np.nan, 31],
    'Column C': [np.nan, np.nan, 100]
})

In [40]:
df2

,Column A,Column B,Column C
0,1.0,2.0,NaN
1,NaN,NaN,NaN
2,30.0,31.0,100.0


In [41]:
df.dropna(how='any')  # default behavior

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


In [80]:
df.dropna(how='all')# drop the col o rows that have all the values as null

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


You can also use the thresh parameter to indicate a threshold (a minimum number) of non-null values for the row/column to be kept:

In [42]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [81]:
df.dropna(thresh=3)# drop col or rowa with 3 or more null values

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34


In [44]:
df.dropna(thresh=3, axis='columns')

,Column B,Column C,Column D
0,2.0,NaN,5
1,8.0,9.0,8
2,31.0,32.0,34
3,NaN,100.0,110


## Filling null values
Sometimes instead than dropping the null values, we might need to replace them with some other value. This highly depends on your context and the dataset you're currently working. Sometimes a nan can be replaced with a 0, sometimes it can be replaced with the mean of the sample, and some other times you can take the closest value. Again, it depends on the context. We'll show you the different methods and mechanisms and you can then apply them to your own problem.

In [45]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

### Filling nulls with a arbitrary value

In [82]:
s.fillna(0) # fill the null values as 0

0    1.0
1    2.0
2    3.0
3    0.0
4    0.0
5    4.0
dtype: float64

In [48]:
s.fillna(s.mean())

0    1.0
1    2.0
2    3.0
3    2.5
4    2.5
5    4.0
dtype: float64

In [49]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

### Filling nulls with contiguous (close) values

The method argument is used to fill null values with other values close to that null one:

In [51]:
s.fillna(method='ffill') # null as the same last value

0    1.0
1    2.0
2    3.0
3    3.0
4    3.0
5    4.0
dtype: float64

In [52]:
s.fillna(method='bfill') # null as the next value

0    1.0
1    2.0
2    3.0
3    4.0
4    4.0
5    4.0
dtype: float64

This can still leave null values at the extremes of the Series/DataFrame:

In [53]:
pd.Series([np.nan, 3, np.nan, 9]).fillna(method='ffill')

0    NaN
1    3.0
2    3.0
3    9.0
dtype: float64

In [54]:
pd.Series([1, np.nan, 3, np.nan, np.nan]).fillna(method='bfill')

0    1.0
1    3.0
2    3.0
3    NaN
4    NaN
dtype: float64

## Filling null values on DataFrames
The fillna method also works on DataFrames, and it works similarly. The main differences are that you can specify the axis (as usual, rows or columns) to use to fill the values (specially for methods) and that you have more control on the values passed:

In [55]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [58]:
df.fillna({'Column A': 0, 'Column B': 99, 'Column C': df['Column C'].mean()}) #fill missing data with this value

,Column A,Column B,Column C,Column D
0,1.0,2.0,47.0,5
1,0.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,0.0,99.0,100.0,110


In [84]:
df.fillna(method='ffill', axis=0) # row, the las value

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,1.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,30.0,31.0,100.0,110


In [85]:
df.fillna(method='ffill', axis=1) # fill the value with the value of the last col

,Column A,Column B,Column C,Column D
0,1.0,2.0,2.0,5.0
1,NaN,8.0,9.0,8.0
2,30.0,31.0,32.0,34.0
3,NaN,NaN,100.0,110.0


## Checking if there are NAs

The question is: Does this Series or DataFrame contain any missing value? The answer should be yes or no: True or False. How can you verify it?

**Example 1: Checking the length**

If there are missing values, s.dropna() will have less elements than s:

In [62]:
s.dropna().count()

4

In [63]:
missing_values = len(s.dropna()) != len(s)
missing_values

True

There's also a count method, that excludes nans from its result:

In [64]:
len(s)

6

In [65]:
s.count()

4

So we could just do:

In [67]:
missing_values = s.count() != len(s)
missing_values

True

### More Pythonic solution any

The methods any and all check if either there's any True value in a Series or all the values are True. They work in the same way as in Python:

In [69]:
pd.Series([True, False, False]).any()

True

In [70]:
pd.Series([True, False, False]).all()

False

In [71]:
pd.Series([True, True, True]).all()

True


The isnull() method returned a Boolean Series with True values wherever there was a nan:

In [73]:
s.isnull()

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool


So we can just use the any method with the boolean array returned:

In [74]:
pd.Series([1, np.nan]).isnull().any()

True

In [75]:
pd.Series([1, 2]).isnull().any()

False

In [76]:
s.isnull().any()

True

A more strict version would check only the values of the Series:

In [77]:
s.isnull().values

array([False, False, False,  True,  True, False])

In [83]:
s.isnull().values.any()

True